Libraries required. 

In [3]:
#from __future__ import print_function
import numpy as np  # array manipulations
import tensorflow as tf # deep neural network
import pickle # load the dataset

Load the dataset. 

In [4]:
pickle_file = 'notMNIST.pickle'
image_size = 28
num_labels = 10
with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  
  print('Training set :', train_dataset.shape, train_labels.shape)
  print('Validation set :', valid_dataset.shape, valid_labels.shape)
  print('Test set :', test_dataset.shape, test_labels.shape)

Training set : (200000, 28, 28) (200000,)
Validation set : (10000, 28, 28) (10000,)
Test set : (10000, 28, 28) (10000,)


Represent the data as row vectors.

In [5]:
def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


Function to create the hidden layer.

In [6]:
def create_neurons(dataset):
    #the basic equation to solve : Y = WX + B 
    layer_1 = tf.add(tf.matmul(dataset, weights['hl']), biases['hl'])
    layer_1 = tf.nn.relu(layer_1)

    final = tf.add(tf.matmul(layer_1, weights['out-layer']), biases['out-layer'])
    return final

Define the structure of the network.

In [7]:
batch_size = 512 # the size of the mini batch trained in each step. Hyper parameter.
learning_rate = 0.1 #hyper parameter.
network = tf.Graph()
with network.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    num_neurons = 1024
    
    # the weights and biases of the hidden layer.
    weights = { 'hl' : tf.Variable(tf.truncated_normal([image_size * image_size, num_neurons])) ,
                'out-layer' : tf.Variable(tf.truncated_normal([num_neurons, num_labels])) }
    biases = { 'hl' : tf.Variable(tf.zeros([num_neurons])) ,
                 'out-layer' : tf.Variable(tf.zeros([num_labels]))
               }
    
    train_logits = create_neurons(tf_train_dataset)
    # the loss function.
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=train_logits))
    #optimizer
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    #logits for validation and test.
    valid_logits = tf.nn.softmax(create_neurons(tf_valid_dataset))
    test_logits = tf.nn.softmax(create_neurons(tf_test_dataset))

Function to determine the accuracy.

In [8]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Train the network.

In [9]:
num_steps = 1501 #hyper parameter.

with tf.Session(graph=network) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}

    _, l, predictions = session.run([optimizer, loss, train_logits], feed_dict=feed_dict)

    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_logits.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_logits.eval(), test_labels))

Initialized
Minibatch loss at step 0: 376.744385
Minibatch accuracy: 10.7%
Validation accuracy: 23.8%
Minibatch loss at step 500: 19.626358
Minibatch accuracy: 79.3%
Validation accuracy: 80.8%
Minibatch loss at step 1000: 10.491993
Minibatch accuracy: 84.6%
Validation accuracy: 82.3%
Minibatch loss at step 1500: 6.604102
Minibatch accuracy: 86.5%
Validation accuracy: 83.4%
Test accuracy: 90.4%
